# 上网业务——手机上网整体满意度

In [1]:
from sklearnex import patch_sklearn, config_context
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier,StackingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import missingno as msno
from time import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei']

##  预处理数据导入

In [3]:
train = pd.read_excel("./output/check/0.5/手机上网整体满意度.xlsx")

In [4]:
train_data = pd.DataFrame.copy(train, deep=True)
train_data

,用户,手机上网整体满意度,居民小区,办公室,商业街,地铁,高铁,网络信号差/没有信号,显示有信号上不了网,上网过程中网络时断时续或时快时慢,手机上网速度慢,看视频卡顿,打开网页或APP图片慢,打游戏延时大,下载速度慢,手机支付较慢,全部都卡顿,全部网页或APP都慢,微信,淘宝
0,用户30,1,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
1,用户32,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,用户64,1,1,1,1,1,1,1,1,0,1,1,0,0,0,0,1,0,0,0
3,用户95,1,1,1,0,0,0,1,0,0,1,1,1,1,1,0,0,1,0,0
4,用户156,1,0,1,1,1,0,0,1,0,1,1,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6074,用户7006,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6075,用户7013,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6076,用户7015,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6077,用户7016,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
train_data=train_data.drop(['用户'],axis=1)
train_data

,手机上网整体满意度,居民小区,办公室,商业街,地铁,高铁,网络信号差/没有信号,显示有信号上不了网,上网过程中网络时断时续或时快时慢,手机上网速度慢,看视频卡顿,打开网页或APP图片慢,打游戏延时大,下载速度慢,手机支付较慢,全部都卡顿,全部网页或APP都慢,微信,淘宝
0,1,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,0,1,1,0,0,0,0,1,0,0,0
3,1,1,1,0,0,0,1,0,0,1,1,1,1,1,0,0,1,0,0
4,1,0,1,1,1,0,0,1,0,1,1,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6074,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6075,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6076,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6077,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 数据划分

In [6]:
x = train_data.drop(['手机上网整体满意度'],axis=1)
x

,居民小区,办公室,商业街,地铁,高铁,网络信号差/没有信号,显示有信号上不了网,上网过程中网络时断时续或时快时慢,手机上网速度慢,看视频卡顿,打开网页或APP图片慢,打游戏延时大,下载速度慢,手机支付较慢,全部都卡顿,全部网页或APP都慢,微信,淘宝
0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,0,1,1,0,0,0,0,1,0,0,0
3,1,1,0,0,0,1,0,0,1,1,1,1,1,0,0,1,0,0
4,0,1,1,1,0,0,1,0,1,1,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6074,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6076,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6077,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
y = train_data["手机上网整体满意度"]
y

0        1
1        1
2        1
3        1
4        1
        ..
6074    10
6075    10
6076    10
6077    10
6078    10
Name: 手机上网整体满意度, Length: 6079, dtype: int64

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [11]:
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)

## 模型训练

In [9]:
val_set = pd.DataFrame()
    
def eval(model):
  
    start = time()
    pred = model.predict(x_val)
    end = time()        
    temp = pd.DataFrame(
            {   
                "Acc":("%0.3f" % (accuracy_score(y_val,pred))),
                "MAE":("%0.3f" % (mean_absolute_error(y_val,pred))),
                "MSE":("%0.3f" % (mean_squared_error(y_val,pred))),
                "Latency":("%0.1fms" % ((end-start)*1000))
            }, index=[(str(model).split("(")[0])]
    )
    val = pd.concat([val_set,temp])
    
    return val
        

In [10]:
rc = RidgeClassifier()
parameters = {
    "alpha":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    "max_iter":[10, 20, 50],
    "normalize":[True,False]
}

rc_cv = GridSearchCV(rc, parameters, cv=5)
rc_cv.fit(x_train,y_train)
rc_cv.best_estimator_

RidgeClassifier(alpha=0.1, max_iter=10, normalize=False)

In [11]:
rc_eval = eval(rc_cv.best_estimator_)
rc_eval

,Acc,MAE,MSE,Latency
RidgeClassifier,0.465,1.631,7.623,2.0ms


In [12]:
dt = DecisionTreeClassifier()
parameters = {
    "criterion":['gini', 'entropy','log_loss'],
    "max_depth":[2,4,8,16,32],
    "min_samples_leaf":[2,4,8,16,32],
    "min_samples_split":[2,4,8,16,32]
}

dt_cv = GridSearchCV(dt, parameters, cv=5)
dt_cv.fit(x_train,y_train)
dt_cv.best_estimator_

DecisionTreeClassifier(max_depth=4, min_samples_leaf=2)

In [13]:
dt_eval = eval(dt_cv.best_estimator_)
dt_eval

,Acc,MAE,MSE,Latency
DecisionTreeClassifier,0.479,1.651,7.872,1.0ms


In [14]:
rf = RandomForestClassifier()
parameters = {
    "max_depth":[2,4,8,16,32],
    "n_estimators":[5,50,250,500]
}

rf_cv = GridSearchCV(rf, parameters, cv=5)
rf_cv.fit(x_train,y_train)
rf_cv.best_estimator_

RandomForestClassifier(max_depth=4, n_estimators=500)

In [15]:
rf_eval = eval(rf_cv.best_estimator_)
rf_eval

,Acc,MAE,MSE,Latency
RandomForestClassifier,0.484,1.539,7.066,6.0ms


In [16]:
ada = AdaBoostClassifier()
parameters = {
    "learning_rate":[0.01,0.1,1.0,10,100],
    "n_estimators":[5,50,250,500]
}
with config_context(target_offload="gpu:0"):
    ada_cv = GridSearchCV(ada, parameters, cv=5)
    ada_cv.fit(x_train,y_train)
    ada_cv.best_estimator_

In [17]:
ada_eval = eval(ada_cv.best_estimator_)
ada_eval

,Acc,MAE,MSE,Latency
AdaBoostClassifier,0.472,1.600,7.386,89.8ms


In [19]:
# gb = GradientBoostingClassifier()
# parameters = {
#     "learning_rate":[0.01,0.1,1.0,10,100],
#     "max_depth":[2,4,8,16,32],
#     "n_estimators":[5,25,50,250,500]
# }

# gb_cv = GridSearchCV(gb, parameters, cv=5)
# gb_cv.fit(x_train,y_train)
# gb_cv.best_estimator_

In [18]:
xgb = XGBClassifier(eval_metric='mlogloss')
parameters = {
    "n_estimators":[5,25,50,250,500],
    "max_depth":[2,4,8,16,32],
    "learning_rate":[0.01,0.1,1.0,10,100]
}

xgb_cv = GridSearchCV(xgb, parameters, cv=5)
xgb_cv.fit(x_train,y_train)
xgb_cv.best_estimator_

[10:11:00] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:11:00] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:11:00] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:11:00] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\wor

[10:11:16] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:11:16] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:11:16] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:11:16] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\wor

[10:11:46] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:11:47] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:11:47] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:11:48] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\wor

[10:13:05] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:13:06] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:13:06] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:13:08] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\wor

[10:15:06] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:15:07] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:15:08] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:15:10] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\wor

...

[10:29:14] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:15] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:16] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:17] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\wor

[10:29:28] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:30] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:31] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:33] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\wor

[10:29:44] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:45] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:47] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:29:47] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\wor

[10:30:00] WARNING: C:\Users\dev-admin\croot\xgboost-split_1675120659361\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=250, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [19]:
xgb_eval = eval(xgb_cv.best_estimator_)
xgb_eval

,Acc,MAE,MSE,Latency
XGBClassifier,0.472,1.512,6.611,11.0ms


In [20]:
svc = SVC()
parameters = {
    "C":[0.001,0.01,0.1,1.0,10,100,1000],
    "kernel":["linear","poly","rbf","sigmoid"]
}
with config_context(target_offload="gpu:0"):
    svc_cv = GridSearchCV(svc, parameters, cv=5)
    svc_cv.fit(x_train,y_train)
    svc_cv.best_estimator_

RuntimeError: Device support is not implemented

In [ ]:
svc_eval = eval(svc_cv.best_estimator_)
svc_eval

In [22]:
mlp = MLPClassifier()
parameters = {
    "activation":["identity","logistic","tanh","relu"],
    "hidden_layer_sizes":[(10,),(50,),(100,)],
    "learning_rate":["constant","invscaling","adaptive"],
    "solver":["lbfgs","sgd","adam"]
}

mlp_cv = GridSearchCV(mlp, parameters, cv=5)
mlp_cv.fit(x_train, y_train)
mlp_cv.best_estimator_

MLPClassifier(solver='sgd')

In [1]:
mlp_eval = eval(mlp_cv.best_estimator_)
mlp_eval

NameError: name 'mlp_cv' is not defined

In [23]:
estimators = [("rc",rc_cv.best_estimator_),
              ("dt",dt_cv.best_estimator_),
              ("rf",rf_cv.best_estimator_),
              ("ada",ada_cv.best_estimator_),
#               ("gb",gb_cv.best_estimator_),
              ("xgb",xgb_cv.best_estimator_),
              ("svc",svc_cv.best_estimator_),
              ("mlp",mlp_cv.best_estimator_)]

sc = StackingClassifier(estimators=estimators)
parameters = {
    "passthrough":[True,False]
}

sc_cv = GridSearchCV(sc, parameters, cv=5)
sc_cv.fit(x_train,y_train)
sc_cv.best_estimator_

StackingClassifier(estimators=[('rc',
                                RidgeClassifier(alpha=0.1, max_iter=10,
                                                normalize=True)),
                               ('dt',
                                DecisionTreeClassifier(max_depth=4,
                                                       min_samples_leaf=32)),
                               ('rf',
                                RandomForestClassifier(max_depth=4,
                                                       n_estimators=250)),
                               ('ada',
                                AdaBoostClassifier(learning_rate=0.1,
                                                   n_estimators=250)),
                               ('xgb',
                                XGBClassifier(base_score=0.5, booster='gbtree',
                                              colsample_bylevel=1...
                                              max_delta_step=0, max_depth=2,
              

In [24]:
models = [rc_cv.best_estimator_,
          dt_cv.best_estimator_,
          rf_cv.best_estimator_,
          ada_cv.best_estimator_,
#           gb_cv.best_estimator_,
#           xgb_cv.best_estimator_,
          svc_cv.best_estimator_,
          mlp_cv.best_estimator_,
          sc_cv.best_estimator_]

In [31]:
val_set = pd.DataFrame()
    
for i in models:
        start = time()
        pred = i.predict(x_val)
        end = time()        
        temp = pd.DataFrame(
                {   
                    "Acc":("%0.3f" % (accuracy_score(y_val,pred))),
                    "MAE":("%0.3f" % (mean_absolute_error(y_val,pred))),
                    "MSE":("%0.3f" % (mean_squared_error(y_val,pred))),
                    "Latency":("%0.1fms" % ((end-start)*1000))
                }, index=[(str(i).split("(")[0])]
        )
        val_set = pd.concat([val_set,temp])
val_set

,Acc,MAE,MSE,Latency
RidgeClassifier,0.432,1.941,9.891,4.0ms
DecisionTreeClassifier,0.437,1.801,8.801,2.0ms
RandomForestClassifier,0.431,1.976,10.260,6.0ms
AdaBoostClassifier,0.438,1.862,9.232,159.6ms
XGBClassifier,0.437,1.868,9.343,78.8ms
SVC,0.430,1.946,9.977,75.8ms
MLPClassifier,0.447,1.821,9.046,3.0ms
StackingClassifier,0.447,1.816,8.957,302.2ms


## 预测（StackingClassifier）

In [26]:
test = pd.read_excel("./output/上网测试集.xlsx")

In [27]:
test_data = pd.DataFrame.copy(test, deep=True)
test_data

,用户id,居民小区,办公室,高校,商业街,地铁,农村,高铁,其他，请注明,网络信号差/没有信号,...,GPRS总流量（KB）,是否5G网络客户,客户星级标识,是否不限量套餐到达用户,终端品牌_0,终端品牌_其它品牌,终端品牌_华为,终端品牌_小米科技,终端品牌_步步高,终端品牌_苹果
0,1,1,0,0,0,0,1,1,0,0,...,2215643,1,8,0,0,0,0,0,1,0
1,2,0,0,0,0,0,1,0,0,1,...,23273941,0,8,0,0,0,1,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,2088114,1,5,0,0,0,1,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,3691155,0,5,0,0,0,0,0,0,1
4,5,1,0,0,0,0,0,0,0,1,...,40765275,1,5,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,1606,0,0,0,0,0,0,0,0,0,...,24118010,1,8,0,0,0,1,0,0,0
1606,1607,1,0,0,0,0,0,0,0,0,...,11553961,1,5,0,0,0,1,0,0,0
1607,1608,1,1,0,0,0,0,0,0,1,...,2308312,1,4,0,0,0,0,1,0,0
1608,1609,1,0,0,0,0,0,0,0,0,...,11508141,0,8,0,0,0,1,0,0,0


In [28]:
test_col = col.drop('手机上网整体满意度')

In [29]:
test_data = test_data.loc[:, test_col]
test_data

,网络信号差/没有信号,手机上网速度慢,上网过程中网络时断时续或时快时慢,打开网页或APP图片慢,居民小区,显示有信号上不了网,办公室,看视频卡顿,打游戏延时大,下载速度慢,地铁,全部网页或APP都慢,手机支付较慢,全部都卡顿,商业街,微信,高铁,淘宝,王者荣耀
0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1606,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1607,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0
1608,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
y_pred_test = sr_cv.best_estimator_.predict(test_data)
output = pd.DataFrame({'Id': test['用户id'], '手机上网整体满意度': np.round(y_pred_test, 0)})
output.to_excel('./output/result-手机上网整体满意度.xlsx', index=False)